In [2]:
import numpy as np
import scipy.sparse as sp

np.random.seed(42)
nb_assets = 10
mean_return_assets = np.random.randn(n, 1)
covariance_assets = np.random.randn(n , n)
covariance_assets = covariance_assets.T.dot(covariance_assets)

NameError: name 'n' is not defined